In [1]:
#coding:utf-8
#导入warnings包，利用过滤器来实现忽略警告语句。
import warnings
warnings.filterwarnings('ignore')
#import missingno as msno
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np

from tsfresh import extract_features

# 1、数据准备

In [2]:
#读取数据集
data_train= pd.read_csv('E:/data mining/train.csv')
data_test = pd.read_csv('E:/data mining/testA.csv')

In [3]:
#将训练数据label列单独存储备用
data_train_label = data_train["label"]

In [4]:
#给测试集加一列label，设为-1
data_test['label']=-1
data_test

,id,heartbeat_signals,label
0,100000,"0.9915713654170097,1.0,0.6318163407681274,0.13...",-1
1,100001,"0.6075533139615096,0.5417083883163654,0.340694...",-1
2,100002,"0.9752726292239277,0.6710965234906665,0.686758...",-1
3,100003,"0.9956348033996116,0.9170249621481004,0.521096...",-1
4,100004,"1.0,0.8879490481178918,0.745564725322326,0.531...",-1
...,...,...,...
19995,119995,"1.0,0.8330283177934747,0.6340472606311671,0.63...",-1
19996,119996,"1.0,0.8259705825857048,0.4521053488322387,0.08...",-1
19997,119997,"0.951744840752379,0.9162611283848351,0.6675251...",-1
19998,119998,"0.9276692903808186,0.6771898159607004,0.242906...",-1


In [6]:
#拼接训练集、测试集得到all_data
all_data=data_train.append(data_test,ignore_index=True)
all_data

,id,heartbeat_signals,label
0,0,"0.9912297987616655,0.9435330436439665,0.764677...",0.0
1,1,"0.9714822034884503,0.9289687459588268,0.572932...",0.0
2,2,"1.0,0.9591487564065292,0.7013782792997189,0.23...",2.0
3,3,"0.9757952826275774,0.9340884687738161,0.659636...",0.0
4,4,"0.0,0.055816398940721094,0.26129357194994196,0...",2.0
...,...,...,...
119995,119995,"1.0,0.8330283177934747,0.6340472606311671,0.63...",-1.0
119996,119996,"1.0,0.8259705825857048,0.4521053488322387,0.08...",-1.0
119997,119997,"0.951744840752379,0.9162611283848351,0.6675251...",-1.0
119998,119998,"0.9276692903808186,0.6771898159607004,0.242906...",-1.0


## 2、加入时间步
### 对all_data加入时间步time

In [7]:
#拆分特征，并对特征进行行转列处理
all_heartbeat_df = all_data["heartbeat_signals"].str.split(",", expand=True).stack()
#重设最外层index
all_heartbeat_df = all_heartbeat_df.reset_index()
#将长度相等的level_0作为新的索引
all_heartbeat_df = all_heartbeat_df.set_index("level_0")
#删除索引名
all_heartbeat_df.index.name = None
#重设特征名
all_heartbeat_df.rename(columns={"level_1":"time", 0:"heartbeat_signals"}, inplace=True)
#转换类型压缩数据
all_heartbeat_df["heartbeat_signals"] = all_heartbeat_df["heartbeat_signals"].astype(float)

all_heartbeat_df

,time,heartbeat_signals
0,0,0.991230
0,1,0.943533
0,2,0.764677
0,3,0.618571
0,4,0.379632
...,...,...
119999,200,0.000000
119999,201,0.000000
119999,202,0.000000
119999,203,0.000000


### 和原id列按索引拼接

In [8]:
# 将处理后的心电特征加入到训练数据中，同时将训练数据label列单独存储
all_data_label = all_data["label"]
all_data = all_data.drop("label", axis=1)
all_data = all_data.drop("heartbeat_signals", axis=1)
all_data = all_data.join(all_heartbeat_df)

all_data

,id,time,heartbeat_signals
0,0,0,0.991230
0,0,1,0.943533
0,0,2,0.764677
0,0,3,0.618571
0,0,4,0.379632
...,...,...,...
119999,119999,200,0.000000
119999,119999,201,0.000000
119999,119999,202,0.000000
119999,119999,203,0.000000


## 3、tsfresh特征工程

### 对all_data做extract_features,得到all_features

In [9]:
from tsfresh import extract_features
# 特征提取
all_features = extract_features(all_data, column_id='id', column_sort='time')
all_features


Feature Extraction: 100%|███████████████████████████████████████████████████████████| 20/20 [2:01:30<00:00, 364.53s/it]


,heartbeat_signals__variance_larger_than_standard_deviation,heartbeat_signals__has_duplicate_max,heartbeat_signals__has_duplicate_min,heartbeat_signals__has_duplicate,heartbeat_signals__sum_values,heartbeat_signals__abs_energy,heartbeat_signals__mean_abs_change,heartbeat_signals__mean_change,heartbeat_signals__mean_second_derivative_central,heartbeat_signals__median,...,heartbeat_signals__permutation_entropy__dimension_6__tau_1,heartbeat_signals__permutation_entropy__dimension_7__tau_1,heartbeat_signals__query_similarity_count__query_None__threshold_0.0,"heartbeat_signals__matrix_profile__feature_""min""__threshold_0.98","heartbeat_signals__matrix_profile__feature_""max""__threshold_0.98","heartbeat_signals__matrix_profile__feature_""mean""__threshold_0.98","heartbeat_signals__matrix_profile__feature_""median""__threshold_0.98","heartbeat_signals__matrix_profile__feature_""25""__threshold_0.98","heartbeat_signals__matrix_profile__feature_""75""__threshold_0.98",heartbeat_signals__mean_n_absolute_max__number_of_maxima_7
0,0.0,0.0,1.0,1.0,38.927945,18.216197,0.019894,-0.004859,0.000117,0.125531,...,2.500658,2.722686,NaN,6.445546,12.165525,10.246524,10.746992,8.388625,11.484910,0.934368
1,0.0,0.0,1.0,1.0,19.445634,7.705092,0.019952,-0.004762,0.000105,0.030481,...,3.065802,3.224835,NaN,3.209140,12.649111,9.031069,9.437545,6.723180,12.094899,0.868868
2,0.0,0.0,1.0,1.0,21.192974,9.140423,0.009863,-0.004902,0.000101,0.000000,...,1.406001,1.509478,NaN,3.054539,8.246211,7.370478,8.246211,5.966122,8.246211,0.623357
3,0.0,0.0,1.0,1.0,42.113066,15.757623,0.018743,-0.004783,0.000103,0.241397,...,3.534354,3.854177,NaN,3.010557,9.797959,6.331360,6.406440,5.266743,7.091706,0.859643
4,0.0,0.0,1.0,1.0,69.756786,51.229616,0.014514,0.000000,-0.000137,0.000000,...,2.165627,2.323993,NaN,9.181236,13.429784,9.959913,9.516290,9.286013,10.270925,0.938180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,0.0,0.0,1.0,1.0,43.175130,18.967833,0.016106,-0.004902,0.000411,0.205399,...,3.625398,3.843586,NaN,3.687770,8.700294,5.991330,6.323450,4.155558,7.191577,0.714023
119996,0.0,0.0,1.0,1.0,31.030782,14.413244,0.021473,-0.004902,0.000429,0.000000,...,1.955659,2.081946,NaN,10.456465,12.982197,11.338307,11.244766,10.763332,11.762948,0.799713
119997,0.0,0.0,1.0,1.0,31.648623,13.083992,0.017566,-0.004665,0.000087,0.010807,...,2.497097,2.663404,NaN,6.037870,11.661904,9.312119,8.973721,8.064338,10.409977,0.835893
119998,0.0,0.0,1.0,1.0,19.305442,6.700835,0.019937,-0.004547,0.000617,0.000000,...,2.912829,3.021449,NaN,10.350940,15.065584,12.961223,12.887409,12.118259,13.558463,0.718066


包含了 heartbeat_signals 的 789 种常见的时间序列特征

### 按索引取出all_features中的train_features

In [11]:
train_features=all_features[0:100000]
train_features

,heartbeat_signals__variance_larger_than_standard_deviation,heartbeat_signals__has_duplicate_max,heartbeat_signals__has_duplicate_min,heartbeat_signals__has_duplicate,heartbeat_signals__sum_values,heartbeat_signals__abs_energy,heartbeat_signals__mean_abs_change,heartbeat_signals__mean_change,heartbeat_signals__mean_second_derivative_central,heartbeat_signals__median,...,heartbeat_signals__permutation_entropy__dimension_6__tau_1,heartbeat_signals__permutation_entropy__dimension_7__tau_1,heartbeat_signals__query_similarity_count__query_None__threshold_0.0,"heartbeat_signals__matrix_profile__feature_""min""__threshold_0.98","heartbeat_signals__matrix_profile__feature_""max""__threshold_0.98","heartbeat_signals__matrix_profile__feature_""mean""__threshold_0.98","heartbeat_signals__matrix_profile__feature_""median""__threshold_0.98","heartbeat_signals__matrix_profile__feature_""25""__threshold_0.98","heartbeat_signals__matrix_profile__feature_""75""__threshold_0.98",heartbeat_signals__mean_n_absolute_max__number_of_maxima_7
0,0.0,0.0,1.0,1.0,38.927945,18.216197,0.019894,-0.004859,0.000117,0.125531,...,2.500658,2.722686,NaN,6.445546,12.165525,10.246524,10.746992,8.388625,11.484910,0.934368
1,0.0,0.0,1.0,1.0,19.445634,7.705092,0.019952,-0.004762,0.000105,0.030481,...,3.065802,3.224835,NaN,3.209140,12.649111,9.031069,9.437545,6.723180,12.094899,0.868868
2,0.0,0.0,1.0,1.0,21.192974,9.140423,0.009863,-0.004902,0.000101,0.000000,...,1.406001,1.509478,NaN,3.054539,8.246211,7.370478,8.246211,5.966122,8.246211,0.623357
3,0.0,0.0,1.0,1.0,42.113066,15.757623,0.018743,-0.004783,0.000103,0.241397,...,3.534354,3.854177,NaN,3.010557,9.797959,6.331360,6.406440,5.266743,7.091706,0.859643
4,0.0,0.0,1.0,1.0,69.756786,51.229616,0.014514,0.000000,-0.000137,0.000000,...,2.165627,2.323993,NaN,9.181236,13.429784,9.959913,9.516290,9.286013,10.270925,0.938180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.0,0.0,1.0,1.0,63.323449,28.742238,0.023588,-0.004902,0.000794,0.388402,...,3.391830,3.679969,NaN,2.436377,9.591663,5.635231,6.366205,3.596982,7.033638,0.749616
99996,0.0,0.0,1.0,1.0,69.657534,31.866323,0.017373,-0.004543,0.000051,0.421138,...,3.728881,4.095457,NaN,1.415410,7.483315,2.893592,2.684349,2.049241,3.334109,0.873778
99997,0.0,0.0,1.0,1.0,40.897057,16.412857,0.019470,-0.004538,0.000834,0.213306,...,2.996962,3.293562,NaN,5.748652,12.165525,8.524637,7.983410,7.062217,10.081756,0.737609
99998,0.0,0.0,1.0,1.0,42.333303,14.281281,0.017032,-0.004902,0.000013,0.264974,...,3.793512,4.018302,NaN,2.346822,8.246211,4.951374,4.727535,4.069786,5.615282,0.767105


In [13]:
#去除含有NaN值的特征

from tsfresh.utilities.dataframe_functions import impute
#对all_features做impute
impute(all_features)
#对train_features做impute
impute(train_features)

,heartbeat_signals__variance_larger_than_standard_deviation,heartbeat_signals__has_duplicate_max,heartbeat_signals__has_duplicate_min,heartbeat_signals__has_duplicate,heartbeat_signals__sum_values,heartbeat_signals__abs_energy,heartbeat_signals__mean_abs_change,heartbeat_signals__mean_change,heartbeat_signals__mean_second_derivative_central,heartbeat_signals__median,...,heartbeat_signals__permutation_entropy__dimension_6__tau_1,heartbeat_signals__permutation_entropy__dimension_7__tau_1,heartbeat_signals__query_similarity_count__query_None__threshold_0.0,"heartbeat_signals__matrix_profile__feature_""min""__threshold_0.98","heartbeat_signals__matrix_profile__feature_""max""__threshold_0.98","heartbeat_signals__matrix_profile__feature_""mean""__threshold_0.98","heartbeat_signals__matrix_profile__feature_""median""__threshold_0.98","heartbeat_signals__matrix_profile__feature_""25""__threshold_0.98","heartbeat_signals__matrix_profile__feature_""75""__threshold_0.98",heartbeat_signals__mean_n_absolute_max__number_of_maxima_7
0,0.0,0.0,1.0,1.0,38.927945,18.216197,0.019894,-0.004859,0.000117,0.125531,...,2.500658,2.722686,0.0,6.445546,12.165525,10.246524,10.746992,8.388625,11.484910,0.934368
1,0.0,0.0,1.0,1.0,19.445634,7.705092,0.019952,-0.004762,0.000105,0.030481,...,3.065802,3.224835,0.0,3.209140,12.649111,9.031069,9.437545,6.723180,12.094899,0.868868
2,0.0,0.0,1.0,1.0,21.192974,9.140423,0.009863,-0.004902,0.000101,0.000000,...,1.406001,1.509478,0.0,3.054539,8.246211,7.370478,8.246211,5.966122,8.246211,0.623357
3,0.0,0.0,1.0,1.0,42.113066,15.757623,0.018743,-0.004783,0.000103,0.241397,...,3.534354,3.854177,0.0,3.010557,9.797959,6.331360,6.406440,5.266743,7.091706,0.859643
4,0.0,0.0,1.0,1.0,69.756786,51.229616,0.014514,0.000000,-0.000137,0.000000,...,2.165627,2.323993,0.0,9.181236,13.429784,9.959913,9.516290,9.286013,10.270925,0.938180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.0,0.0,1.0,1.0,63.323449,28.742238,0.023588,-0.004902,0.000794,0.388402,...,3.391830,3.679969,0.0,2.436377,9.591663,5.635231,6.366205,3.596982,7.033638,0.749616
99996,0.0,0.0,1.0,1.0,69.657534,31.866323,0.017373,-0.004543,0.000051,0.421138,...,3.728881,4.095457,0.0,1.415410,7.483315,2.893592,2.684349,2.049241,3.334109,0.873778
99997,0.0,0.0,1.0,1.0,40.897057,16.412857,0.019470,-0.004538,0.000834,0.213306,...,2.996962,3.293562,0.0,5.748652,12.165525,8.524637,7.983410,7.062217,10.081756,0.737609
99998,0.0,0.0,1.0,1.0,42.333303,14.281281,0.017032,-0.004902,0.000013,0.264974,...,3.793512,4.018302,0.0,2.346822,8.246211,4.951374,4.727535,4.069786,5.615282,0.767105


In [17]:
#根据特征-标签相关性，进一步筛选特征，得到features_filtered

from tsfresh import select_features
features_filtered = select_features(train_features,data_train_label)
features_filtered


,heartbeat_signals__sum_values,"heartbeat_signals__fft_coefficient__attr_""abs""__coeff_38","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_37","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_36","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_35","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_34","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_33","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_32","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_31","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_30",...,"heartbeat_signals__fft_coefficient__attr_""abs""__coeff_84","heartbeat_signals__fft_coefficient__attr_""imag""__coeff_97","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_90","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_94","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_92","heartbeat_signals__fft_coefficient__attr_""real""__coeff_97","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_75","heartbeat_signals__fft_coefficient__attr_""real""__coeff_88","heartbeat_signals__fft_coefficient__attr_""real""__coeff_92","heartbeat_signals__fft_coefficient__attr_""real""__coeff_83"
0,38.927945,0.660949,1.090709,0.848728,1.168685,0.982133,1.223496,1.236300,1.104172,1.497129,...,0.531883,-0.047438,0.554370,0.307586,0.564596,0.562960,0.591859,0.504124,0.528450,0.473568
1,19.445634,1.718217,1.280923,1.850706,1.460752,1.924501,1.925485,1.715938,2.079957,1.818636,...,0.563590,-0.109579,0.697446,0.398073,0.640969,0.270192,0.224925,0.645082,0.635135,0.297325
2,21.192974,1.814281,1.619051,1.215343,1.787166,2.146987,1.686190,1.540137,2.291031,2.403422,...,0.712487,-0.074042,0.321703,0.390386,0.716929,0.316524,0.422077,0.722742,0.680590,0.383754
3,42.113066,2.109550,0.619634,2.366413,2.071539,1.000340,2.728281,1.391727,2.017176,2.610492,...,0.601499,-0.184248,0.564669,0.623353,0.466980,0.651774,0.308915,0.550097,0.466904,0.494024
4,69.756786,0.194549,0.348882,0.092119,0.653924,0.231422,1.080003,0.711244,1.357904,1.237998,...,0.015292,0.070505,0.065835,0.051780,0.092940,0.103773,0.179405,-0.089611,0.091841,0.056867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,63.323449,0.840651,1.186210,1.396236,0.417221,2.036034,1.659054,0.500584,1.693545,0.859932,...,0.779955,0.005525,0.486013,0.273372,0.705386,0.602898,0.447929,0.474844,0.564266,0.133969
99996,69.657534,1.557787,1.393960,0.989147,1.611333,1.793044,1.092325,0.507138,1.763940,2.677643,...,0.539489,0.114670,0.579498,0.417226,0.270110,0.556596,0.703258,0.462312,0.269719,0.539236
99997,40.897057,0.469758,1.000355,0.706395,1.190514,0.674603,1.632769,0.229008,2.027802,0.302457,...,0.282597,-0.474629,0.460647,0.478341,0.527891,0.904111,0.728529,0.178410,0.500813,0.773985
99998,42.333303,0.992948,1.354894,2.238589,1.237608,1.325212,2.785515,1.918571,0.814167,2.613950,...,0.594252,-0.162106,0.694276,0.681025,0.357196,0.498088,0.433297,0.406154,0.324771,0.340727


最后，剩下了709个时序特征

In [26]:
#手动把all_features中这些特征也筛选出来
all_features=all_features[features_filtered.columns]
all_features

,heartbeat_signals__sum_values,"heartbeat_signals__fft_coefficient__attr_""abs""__coeff_38","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_37","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_36","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_35","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_34","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_33","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_32","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_31","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_30",...,"heartbeat_signals__fft_coefficient__attr_""abs""__coeff_84","heartbeat_signals__fft_coefficient__attr_""imag""__coeff_97","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_90","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_94","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_92","heartbeat_signals__fft_coefficient__attr_""real""__coeff_97","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_75","heartbeat_signals__fft_coefficient__attr_""real""__coeff_88","heartbeat_signals__fft_coefficient__attr_""real""__coeff_92","heartbeat_signals__fft_coefficient__attr_""real""__coeff_83"
0,38.927945,0.660949,1.090709,0.848728,1.168685,0.982133,1.223496,1.236300,1.104172,1.497129,...,0.531883,-0.047438,0.554370,0.307586,0.564596,0.562960,0.591859,0.504124,0.528450,0.473568
1,19.445634,1.718217,1.280923,1.850706,1.460752,1.924501,1.925485,1.715938,2.079957,1.818636,...,0.563590,-0.109579,0.697446,0.398073,0.640969,0.270192,0.224925,0.645082,0.635135,0.297325
2,21.192974,1.814281,1.619051,1.215343,1.787166,2.146987,1.686190,1.540137,2.291031,2.403422,...,0.712487,-0.074042,0.321703,0.390386,0.716929,0.316524,0.422077,0.722742,0.680590,0.383754
3,42.113066,2.109550,0.619634,2.366413,2.071539,1.000340,2.728281,1.391727,2.017176,2.610492,...,0.601499,-0.184248,0.564669,0.623353,0.466980,0.651774,0.308915,0.550097,0.466904,0.494024
4,69.756786,0.194549,0.348882,0.092119,0.653924,0.231422,1.080003,0.711244,1.357904,1.237998,...,0.015292,0.070505,0.065835,0.051780,0.092940,0.103773,0.179405,-0.089611,0.091841,0.056867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,43.175130,1.776937,0.211527,1.986940,0.393550,1.693620,1.139395,1.459990,1.734535,1.025180,...,0.546742,-0.060254,0.507950,0.560192,0.541534,0.249750,0.608796,0.455444,0.535306,0.268471
119996,31.030782,1.451045,2.483726,1.105440,1.979721,2.821799,0.475276,2.782573,2.827882,0.520034,...,0.491662,0.016413,0.480380,0.459172,0.363756,0.427028,0.544692,0.754834,0.361866,0.536087
119997,31.648623,2.141301,0.546706,2.340499,1.362651,1.942634,2.043679,0.994065,2.248144,1.007128,...,0.529880,0.001012,0.768960,0.834159,0.672114,0.520215,0.341519,0.713419,0.664354,0.370047
119998,19.305442,0.221708,2.355288,1.051282,1.742370,2.164058,0.435583,2.649994,1.190594,2.328580,...,0.527500,-0.103574,0.521222,0.426435,0.636887,0.446365,0.551442,0.503703,0.635246,0.258394


In [27]:
#把all_data_label列加回来，按照all_data_label=-1拆分出测试集、训练集
all_features=all_features.join(all_data_label)
all_features



#重命名训练测试集的列名，为训练做准备

,heartbeat_signals__sum_values,"heartbeat_signals__fft_coefficient__attr_""abs""__coeff_38","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_37","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_36","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_35","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_34","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_33","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_32","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_31","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_30",...,"heartbeat_signals__fft_coefficient__attr_""imag""__coeff_97","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_90","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_94","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_92","heartbeat_signals__fft_coefficient__attr_""real""__coeff_97","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_75","heartbeat_signals__fft_coefficient__attr_""real""__coeff_88","heartbeat_signals__fft_coefficient__attr_""real""__coeff_92","heartbeat_signals__fft_coefficient__attr_""real""__coeff_83",label
0,38.927945,0.660949,1.090709,0.848728,1.168685,0.982133,1.223496,1.236300,1.104172,1.497129,...,-0.047438,0.554370,0.307586,0.564596,0.562960,0.591859,0.504124,0.528450,0.473568,0.0
1,19.445634,1.718217,1.280923,1.850706,1.460752,1.924501,1.925485,1.715938,2.079957,1.818636,...,-0.109579,0.697446,0.398073,0.640969,0.270192,0.224925,0.645082,0.635135,0.297325,0.0
2,21.192974,1.814281,1.619051,1.215343,1.787166,2.146987,1.686190,1.540137,2.291031,2.403422,...,-0.074042,0.321703,0.390386,0.716929,0.316524,0.422077,0.722742,0.680590,0.383754,2.0
3,42.113066,2.109550,0.619634,2.366413,2.071539,1.000340,2.728281,1.391727,2.017176,2.610492,...,-0.184248,0.564669,0.623353,0.466980,0.651774,0.308915,0.550097,0.466904,0.494024,0.0
4,69.756786,0.194549,0.348882,0.092119,0.653924,0.231422,1.080003,0.711244,1.357904,1.237998,...,0.070505,0.065835,0.051780,0.092940,0.103773,0.179405,-0.089611,0.091841,0.056867,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,43.175130,1.776937,0.211527,1.986940,0.393550,1.693620,1.139395,1.459990,1.734535,1.025180,...,-0.060254,0.507950,0.560192,0.541534,0.249750,0.608796,0.455444,0.535306,0.268471,-1.0
119996,31.030782,1.451045,2.483726,1.105440,1.979721,2.821799,0.475276,2.782573,2.827882,0.520034,...,0.016413,0.480380,0.459172,0.363756,0.427028,0.544692,0.754834,0.361866,0.536087,-1.0
119997,31.648623,2.141301,0.546706,2.340499,1.362651,1.942634,2.043679,0.994065,2.248144,1.007128,...,0.001012,0.768960,0.834159,0.672114,0.520215,0.341519,0.713419,0.664354,0.370047,-1.0
119998,19.305442,0.221708,2.355288,1.051282,1.742370,2.164058,0.435583,2.649994,1.190594,2.328580,...,-0.103574,0.521222,0.426435,0.636887,0.446365,0.551442,0.503703,0.635246,0.258394,-1.0


In [35]:
test_features=all_features.loc[all_features['label'].isin([-1.0])]
test_features

,heartbeat_signals__sum_values,"heartbeat_signals__fft_coefficient__attr_""abs""__coeff_38","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_37","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_36","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_35","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_34","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_33","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_32","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_31","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_30",...,"heartbeat_signals__fft_coefficient__attr_""imag""__coeff_97","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_90","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_94","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_92","heartbeat_signals__fft_coefficient__attr_""real""__coeff_97","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_75","heartbeat_signals__fft_coefficient__attr_""real""__coeff_88","heartbeat_signals__fft_coefficient__attr_""real""__coeff_92","heartbeat_signals__fft_coefficient__attr_""real""__coeff_83",label
100000,19.229863,2.381214,0.832151,2.509869,1.082112,2.517858,1.656104,2.257162,2.213421,1.815374,...,-0.040576,0.485441,0.472059,0.448018,0.449347,0.479950,0.480448,0.442279,0.355992,-1.0
100001,84.298932,0.987660,0.856174,0.616261,0.293339,0.191558,0.528684,1.010080,1.478182,1.713876,...,0.010074,0.272897,0.247538,0.286948,0.143829,0.189416,0.124293,0.154624,0.077530,-1.0
100002,47.789921,0.696393,1.165387,1.004378,0.951231,1.542114,0.946219,1.673430,1.445220,1.118439,...,-0.159505,0.418298,0.566628,0.849684,0.950851,0.779324,0.439255,0.839315,0.454957,-1.0
100003,47.069011,3.137668,0.044897,3.392946,3.054217,0.726293,3.582653,2.414946,1.257669,3.188068,...,0.366949,0.891690,0.214585,0.927562,0.648872,0.730178,0.606528,0.830105,0.662320,-1.0
100004,24.899397,0.496010,1.401020,0.536501,1.712592,1.044629,1.533405,1.330258,1.251771,1.441028,...,-0.129700,0.578560,0.783258,0.480598,0.485003,0.667111,0.594234,0.447980,0.511133,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,43.175130,1.776937,0.211527,1.986940,0.393550,1.693620,1.139395,1.459990,1.734535,1.025180,...,-0.060254,0.507950,0.560192,0.541534,0.249750,0.608796,0.455444,0.535306,0.268471,-1.0
119996,31.030782,1.451045,2.483726,1.105440,1.979721,2.821799,0.475276,2.782573,2.827882,0.520034,...,0.016413,0.480380,0.459172,0.363756,0.427028,0.544692,0.754834,0.361866,0.536087,-1.0
119997,31.648623,2.141301,0.546706,2.340499,1.362651,1.942634,2.043679,0.994065,2.248144,1.007128,...,0.001012,0.768960,0.834159,0.672114,0.520215,0.341519,0.713419,0.664354,0.370047,-1.0
119998,19.305442,0.221708,2.355288,1.051282,1.742370,2.164058,0.435583,2.649994,1.190594,2.328580,...,-0.103574,0.521222,0.426435,0.636887,0.446365,0.551442,0.503703,0.635246,0.258394,-1.0


In [32]:
train_features=all_features.loc[all_features['label'].isin([0.0,1.0,2.0,3.0])]
train_features

,heartbeat_signals__sum_values,"heartbeat_signals__fft_coefficient__attr_""abs""__coeff_38","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_37","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_36","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_35","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_34","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_33","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_32","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_31","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_30",...,"heartbeat_signals__fft_coefficient__attr_""imag""__coeff_97","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_90","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_94","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_92","heartbeat_signals__fft_coefficient__attr_""real""__coeff_97","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_75","heartbeat_signals__fft_coefficient__attr_""real""__coeff_88","heartbeat_signals__fft_coefficient__attr_""real""__coeff_92","heartbeat_signals__fft_coefficient__attr_""real""__coeff_83",label
0,38.927945,0.660949,1.090709,0.848728,1.168685,0.982133,1.223496,1.236300,1.104172,1.497129,...,-0.047438,0.554370,0.307586,0.564596,0.562960,0.591859,0.504124,0.528450,0.473568,0.0
1,19.445634,1.718217,1.280923,1.850706,1.460752,1.924501,1.925485,1.715938,2.079957,1.818636,...,-0.109579,0.697446,0.398073,0.640969,0.270192,0.224925,0.645082,0.635135,0.297325,0.0
2,21.192974,1.814281,1.619051,1.215343,1.787166,2.146987,1.686190,1.540137,2.291031,2.403422,...,-0.074042,0.321703,0.390386,0.716929,0.316524,0.422077,0.722742,0.680590,0.383754,2.0
3,42.113066,2.109550,0.619634,2.366413,2.071539,1.000340,2.728281,1.391727,2.017176,2.610492,...,-0.184248,0.564669,0.623353,0.466980,0.651774,0.308915,0.550097,0.466904,0.494024,0.0
4,69.756786,0.194549,0.348882,0.092119,0.653924,0.231422,1.080003,0.711244,1.357904,1.237998,...,0.070505,0.065835,0.051780,0.092940,0.103773,0.179405,-0.089611,0.091841,0.056867,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,63.323449,0.840651,1.186210,1.396236,0.417221,2.036034,1.659054,0.500584,1.693545,0.859932,...,0.005525,0.486013,0.273372,0.705386,0.602898,0.447929,0.474844,0.564266,0.133969,0.0
99996,69.657534,1.557787,1.393960,0.989147,1.611333,1.793044,1.092325,0.507138,1.763940,2.677643,...,0.114670,0.579498,0.417226,0.270110,0.556596,0.703258,0.462312,0.269719,0.539236,2.0
99997,40.897057,0.469758,1.000355,0.706395,1.190514,0.674603,1.632769,0.229008,2.027802,0.302457,...,-0.474629,0.460647,0.478341,0.527891,0.904111,0.728529,0.178410,0.500813,0.773985,3.0
99998,42.333303,0.992948,1.354894,2.238589,1.237608,1.325212,2.785515,1.918571,0.814167,2.613950,...,-0.162106,0.694276,0.681025,0.357196,0.498088,0.433297,0.406154,0.324771,0.340727,2.0


In [36]:
test_features=test_features.drop("label", axis=1)
test_features

,heartbeat_signals__sum_values,"heartbeat_signals__fft_coefficient__attr_""abs""__coeff_38","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_37","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_36","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_35","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_34","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_33","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_32","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_31","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_30",...,"heartbeat_signals__fft_coefficient__attr_""abs""__coeff_84","heartbeat_signals__fft_coefficient__attr_""imag""__coeff_97","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_90","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_94","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_92","heartbeat_signals__fft_coefficient__attr_""real""__coeff_97","heartbeat_signals__fft_coefficient__attr_""abs""__coeff_75","heartbeat_signals__fft_coefficient__attr_""real""__coeff_88","heartbeat_signals__fft_coefficient__attr_""real""__coeff_92","heartbeat_signals__fft_coefficient__attr_""real""__coeff_83"
100000,19.229863,2.381214,0.832151,2.509869,1.082112,2.517858,1.656104,2.257162,2.213421,1.815374,...,0.563470,-0.040576,0.485441,0.472059,0.448018,0.449347,0.479950,0.480448,0.442279,0.355992
100001,84.298932,0.987660,0.856174,0.616261,0.293339,0.191558,0.528684,1.010080,1.478182,1.713876,...,0.037307,0.010074,0.272897,0.247538,0.286948,0.143829,0.189416,0.124293,0.154624,0.077530
100002,47.789921,0.696393,1.165387,1.004378,0.951231,1.542114,0.946219,1.673430,1.445220,1.118439,...,0.738423,-0.159505,0.418298,0.566628,0.849684,0.950851,0.779324,0.439255,0.839315,0.454957
100003,47.069011,3.137668,0.044897,3.392946,3.054217,0.726293,3.582653,2.414946,1.257669,3.188068,...,0.273142,0.366949,0.891690,0.214585,0.927562,0.648872,0.730178,0.606528,0.830105,0.662320
100004,24.899397,0.496010,1.401020,0.536501,1.712592,1.044629,1.533405,1.330258,1.251771,1.441028,...,0.644046,-0.129700,0.578560,0.783258,0.480598,0.485003,0.667111,0.594234,0.447980,0.511133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,43.175130,1.776937,0.211527,1.986940,0.393550,1.693620,1.139395,1.459990,1.734535,1.025180,...,0.546742,-0.060254,0.507950,0.560192,0.541534,0.249750,0.608796,0.455444,0.535306,0.268471
119996,31.030782,1.451045,2.483726,1.105440,1.979721,2.821799,0.475276,2.782573,2.827882,0.520034,...,0.491662,0.016413,0.480380,0.459172,0.363756,0.427028,0.544692,0.754834,0.361866,0.536087
119997,31.648623,2.141301,0.546706,2.340499,1.362651,1.942634,2.043679,0.994065,2.248144,1.007128,...,0.529880,0.001012,0.768960,0.834159,0.672114,0.520215,0.341519,0.713419,0.664354,0.370047
119998,19.305442,0.221708,2.355288,1.051282,1.742370,2.164058,0.435583,2.649994,1.190594,2.328580,...,0.527500,-0.103574,0.521222,0.426435,0.636887,0.446365,0.551442,0.503703,0.635246,0.258394


In [37]:
outputpath='E:/data mining/train_features.csv'
train_features.to_csv(outputpath,sep=',',index=True,header=True)

In [38]:
outputpath1='E:/data mining/test_features.csv'
test_features.to_csv(outputpath1,sep=',',index=True,header=True)